# Conectando ao DB Postegres, para coleta de dados

Pedro H. N. Sobrinho (PSobrinho)

Mail: pedrosobrinho.physics@gmail.com

LinkedIn: https://www.linkedin.com/in/pedro-sobrinho-b84879108/

Instagram: https://www.instagram.com/phnsobrinho/?hl=pt-br


Construído com o objetivo de estudar a conecção e acessoa à dados em bases de dado Postgres. o presente notebook faz parte do estudo de analise, exploração e vizualização de dados, do conjunto 'used_cars_german'. Sendo assim disponibilizado de forma aberta para consulta e estudo por terceiros. 

Observe que:

* Foi utilizada uma base de dados local para treino, isto é, as credenciais de acesso aqui utilizada são locais.

* Os 'Result Sets' gerados foram organizado e convertidos em diferentes 'DataFrames' para cada etapa da posterior análise. Sendo estas:

    * **Distribuição de veículos com base no ano de registro**;
    * **Variações de faixa de preço pelo tipo de veículo**;
    * **Contagem total de veículos à venda conforme tipo de veículo**;
    * **Nº de veículos por marcas**;
    * **Preço médio com base no tipo de veículo, bem como no tipo de caixa de câmbio**;
    * **Preço médio do veículo por tipo de combustível e caixa de câmbio**;
    * **Potência Média por tipo de veículo e caixa de câmbio**.


# Conectando ao BD Postegres

In [0]:
import psycopg2 #pacote para interação com BD Postgres

In [0]:
#Criando a conecção com o banco de dados (local)
conn = psycopg2.connect(host = 'localhost',
                        port = 5432,
                        dbname = 'studies',
                        user = 'youruser',
                        password = 'yourpass')

In [0]:
#criando um cursor para acessar os dados no BD
cur = conn.cursor

# Buscando dados via queries com Pandas

A Ideia aqui é utilizar Queries SQL para buscar dados no BD, e salvar os 'result sets' como arquivos .csv

In [0]:
import pandas as pd #importando Pandas usando o 'alias' pd

**O DataSet original**

In [0]:
df = pd.read_sql('SELECT * FROM usedcars;',
                 conn)

print(df)

               datecrawled                                               name  \
0      2016-03-24 11:52:17                                         Golf_3_1.6   
1      2016-03-24 10:58:45                               A5_Sportback_2.7_Tdi   
2      2016-03-14 12:52:21                       Jeep_Grand_Cherokee_Overland   
3      2016-03-17 16:54:04                                 GOLF_4_1_4__3TÜRER   
4      2016-03-31 17:25:20                     Skoda_Fabia_1.4_TDI_PD_Classic   
5      2016-04-04 17:36:23  BMW_316i___e36_Limousine___Bastlerfahrzeug__Ex...   
6      2016-04-01 20:48:51                        Peugeot_206_CC_110_Platinum   
7      2016-03-21 18:54:38                       VW_Derby_Bj_80__Scheunenfund   
8      2016-04-04 23:42:13               Ford_C___Max_Titanium_1_0_L_EcoBoost   
9      2016-03-17 10:53:50  VW_Golf_4_5_tuerig_zu_verkaufen_mit_Anhaengerk...   
10     2016-03-26 19:54:18                                  Mazda_3_1.6_Sport   
11     2016-04-07 10:06:22  

**Distribuição de veículos com base no ano de registro**

Para esse DataFrame, as informações relevantes são os anos de registro e a contagem de registros associados a cada ano na tabela.

In [0]:
#Começamos por buscar no banco de dados os dados relevantes:
df1 = pd.read_sql("SELECT yearofregistration, COUNT(*) AS yearcount FROM usedcars GROUP BY yearofregistration", 
                  conn)

print(df1)

     yearofregistration  yearcount
0                  1000         21
1                  1111          1
2                  1200          1
3                  1234          2
4                  1300          2
5                  1400          1
6                  1500          2
7                  1602          1
8                  1800          4
9                  1910         46
10                 1915          1
11                 1919          1
12                 1928          1
13                 1929          3
14                 1930          3
15                 1931          2
16                 1933          3
17                 1934          3
18                 1935          3
19                 1936          4
20                 1937          4
21                 1938          3
22                 1939          2
23                 1940          1
24                 1941          1
25                 1942          1
26                 1943          3
27                 1

Note que o ResultSet obtido contém dados que, provavelmente, são equivocados. Logo, vale uma limpeza no DataFrame.

In [0]:
#Na ocasião da preparação desse notebook o ano é 2020, então vamos considerar registros feitos nos últimos 120 anos
#(considerando que a primeira patente de automóvel fez 130 anos em 2016), isto é vamos considerar apenas anos de registro
#que estão entre 1900 e 2020:

df1_clean = df1.loc[(df1['yearofregistration'] >= 1900) & (df1['yearofregistration'] <= 2020)]

df1_clean = df1_clean.reset_index(drop = True)

print(df1_clean)

    yearofregistration  yearcount
0                 1910         46
1                 1915          1
2                 1919          1
3                 1928          1
4                 1929          3
5                 1930          3
6                 1931          2
7                 1933          3
8                 1934          3
9                 1935          3
10                1936          4
11                1937          4
12                1938          3
13                1939          2
14                1940          1
15                1941          1
16                1942          1
17                1943          3
18                1945          1
19                1946          1
20                1947          3
21                1949          2
22                1950         11
23                1951         10
24                1952          4
25                1953         10
26                1954         11
27                1955         14
28            

In [0]:
#Com os dados coletados e limpos, resta salvar o dataset:
df1_clean.to_csv('../data/df1_yearcount.csv', sep = ';', index = False)

Todos os DataFrames a seguir seguirão esse procedimento:
* Coleta de dados via query;
* Limpeza de dados, provavelmente, equivocados;
* Fechamento em um arquivo .csv.

Observe que as queries a partir daqui, já considerarão apenas as entradas cujo ano de registro pertence ao período de 1900 a 2020 que definimos para o DataFrame anterior

**Variações de faixa de preço pelo tipo de veículo**

Os dados relevantes são os tipos de veículo e seus respectivos preços

In [0]:
#Coletando dados:
df2 = pd.read_sql('SELECT vehicletype, price FROM usedcars WHERE yearofregistration BETWEEN 1900 AND 2020;',
                 conn)

print(df2)

       vehicletype    price
0             None    480.0
1            coupe  18300.0
2              suv   9800.0
3       kleinwagen   1500.0
4       kleinwagen   3600.0
5        limousine    650.0
6           cabrio   2200.0
7        limousine      0.0
8              bus  14500.0
9       kleinwagen    999.0
10       limousine   2000.0
11           kombi   2799.0
12           kombi    999.0
13           kombi   2500.0
14             suv  17999.0
15      kleinwagen    450.0
16            None    300.0
17      kleinwagen   1750.0
18             bus   7550.0
19             bus   1850.0
20           coupe  10400.0
21       limousine   3699.0
22            None   2900.0
23      kleinwagen    450.0
24       limousine    500.0
25           kombi   2500.0
26            None   5555.0
27       limousine   6900.0
28             bus   1990.0
29      kleinwagen    690.0
...            ...      ...
189228   limousine    699.0
189229  kleinwagen    600.0
189230   limousine  20400.0
189231         bus  

In [0]:
#Neste caso, aparentemente não precisamos fazer uma limpeza, vamos direto para o salvamento:
df2.to_csv('../data/df2_vehicleprice.csv', sep = ';', index = False)

**Contagem total de veículos à venda conforme tipo de veículo**

Os dados relevantes são os tipos de veículos e a contagem de registros associados a cada um destes

In [0]:
#Coletando dados:
df3 = pd.read_sql('SELECT vehicletype, COUNT(*) AS vehiclecount FROM usedcars WHERE yearofregistration BETWEEN 1900 AND 2020 GROUP BY vehicletype;',
                  conn)

print(df3)

  vehicletype  vehiclecount
0      andere          1708
1         bus         15532
2      cabrio         11668
3       coupe          9635
4  kleinwagen         40759
5       kombi         34498
6   limousine         48701
7         suv          7411
8        None         19346


In [0]:
#Salvando DataFrame
df3.to_csv('../data/df3_vehiclecount.csv', sep = ';', index = False)

**Nº de veículos por marcas**

Os dados relevantes são as marcas de veículos e a contagem de registros associados a cada um

In [0]:
#Coletando dados:
df4 = pd.read_sql('SELECT brand, COUNT(*) AS brandcount FROM usedcars WHERE yearofregistration BETWEEN 1900 AND 2020 GROUP BY brand',
                 conn)

print(df4)

             brand  brandcount
0       alfa_romeo        1184
1             audi       16674
2              bmw       20540
3        chevrolet         923
4         chrysler         745
5          citroen        2633
6            dacia         455
7           daewoo         283
8         daihatsu         400
9             fiat        4935
10            ford       13090
11           honda        1429
12         hyundai        1830
13          jaguar         303
14            jeep         406
15             kia        1280
16            lada          98
17          lancia         243
18      land_rover         388
19           mazda        2941
20   mercedes_benz       17925
21            mini        1708
22      mitsubishi        1519
23          nissan        2564
24            opel       20416
25         peugeot        5695
26         porsche        1141
27         renault        9144
28           rover         262
29            saab         261
30            seat        3531
31      

In [0]:
#Este também aparentemente não precisa de limpeza.
#Salvando DataFrame:

df4.to_csv('../data/df4_brandcount.csv', sep = ';', index = False)

**Preço médio com base no tipo de veículo, bem como no tipo de caixa de câmbio**

Os dados relevantes são os ipos de veículos, suas respectivaas caixas de câmbio e preços

In [0]:
#Coletando dados:
df5 = pd.read_sql('SELECT vehicletype, gearbox, price FROM usedcars WHERE yearofregistration BETWEEN 1900 AND 2020;',
                 conn)

print(df5)

       vehicletype    gearbox    price
0             None    manuell    480.0
1            coupe    manuell  18300.0
2              suv  automatik   9800.0
3       kleinwagen    manuell   1500.0
4       kleinwagen    manuell   3600.0
5        limousine    manuell    650.0
6           cabrio    manuell   2200.0
7        limousine    manuell      0.0
8              bus    manuell  14500.0
9       kleinwagen    manuell    999.0
10       limousine    manuell   2000.0
11           kombi    manuell   2799.0
12           kombi    manuell    999.0
13           kombi    manuell   2500.0
14             suv    manuell  17999.0
15      kleinwagen       None    450.0
16            None       None    300.0
17      kleinwagen  automatik   1750.0
18             bus    manuell   7550.0
19             bus    manuell   1850.0
20           coupe    manuell  10400.0
21       limousine  automatik   3699.0
22            None    manuell   2900.0
23      kleinwagen    manuell    450.0
24       limousine    man

In [0]:
#Salvando DataFrame
df5.to_csv('../data/df5_vehicle_gearbox_price.csv', sep = ';', index = False)

 **Preço médio do veículo por tipo de combustível e caixa de câmbio**

Os dados relevantes são os tipos de veículos, seus tipos de caixa de câmbio e tipos de combustível

In [0]:
#Coletando dados:
df6 = pd.read_sql('SELECT fueltype, gearbox, price FROM usedcars WHERE yearofregistration BETWEEN 1900 AND 2020;',
                 conn)

print(df6)

       fueltype    gearbox    price
0        benzin    manuell    480.0
1        diesel    manuell  18300.0
2        diesel  automatik   9800.0
3        benzin    manuell   1500.0
4        diesel    manuell   3600.0
5        benzin    manuell    650.0
6        benzin    manuell   2200.0
7        benzin    manuell      0.0
8        benzin    manuell  14500.0
9          None    manuell    999.0
10       benzin    manuell   2000.0
11       diesel    manuell   2799.0
12       benzin    manuell    999.0
13         None    manuell   2500.0
14       diesel    manuell  17999.0
15       benzin       None    450.0
16       benzin       None    300.0
17       benzin  automatik   1750.0
18       diesel    manuell   7550.0
19       benzin    manuell   1850.0
20       benzin    manuell  10400.0
21       benzin  automatik   3699.0
22       benzin    manuell   2900.0
23       benzin    manuell    450.0
24       benzin    manuell    500.0
25       diesel  automatik   2500.0
26         None    manuell  

In [0]:
#Salvando dados:
df6.to_csv('../data/df6_fueltype_gearbox_price.csv', sep = ';', index = False)

**Potência Média por tipo de veículo e caixa de câmbio**

Os dados relevantes são os tipos de veículos, suas respectivas caixas de câmbio e potências

In [0]:
#Coletando dados:
df7 = pd.read_sql('SELECT vehicletype, gearbox, powerps FROM usedcars WHERE yearofregistration BETWEEN 1900 AND 2020;',
                 conn)

print(df7)

       vehicletype    gearbox  powerps
0             None    manuell      0.0
1            coupe    manuell    190.0
2              suv  automatik    163.0
3       kleinwagen    manuell     75.0
4       kleinwagen    manuell     69.0
5        limousine    manuell    102.0
6           cabrio    manuell    109.0
7        limousine    manuell     50.0
8              bus    manuell    125.0
9       kleinwagen    manuell    101.0
10       limousine    manuell    105.0
11           kombi    manuell    140.0
12           kombi    manuell    115.0
13           kombi    manuell    131.0
14             suv    manuell    190.0
15      kleinwagen       None      0.0
16            None       None     60.0
17      kleinwagen  automatik     75.0
18             bus    manuell    136.0
19             bus    manuell    102.0
20           coupe    manuell    160.0
21       limousine  automatik    231.0
22            None    manuell     90.0
23      kleinwagen    manuell     50.0
24       limousine    man

In [0]:
#Salvando DataFrame
df7.to_csv('../data/df7_vehicle_gearbox_price.csv', sep = ';', index = False)

In [0]:
conn.close()